Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

## Have Fun!

In [85]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import functools
from icecream import ic
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)

In [86]:
CVS_FILE = "cities/italy.csv"

In [87]:
CITIES = pd.read_csv(CVS_FILE, header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [88]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def tsp_cost(tsp):
    # assert tsp[0] == tsp[-1]
    # assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Firt Greedy

In [89]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

INFO:root:result: Found a path of 46 steps, total length 4436.03km


## Second Greedy

In [90]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

In [91]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]
visited = set()
edges = set()

In [92]:
shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))
visited |= shortest[0]
visited
edges |= {tuple(shortest[0])}
segments = [s for s in segments if not cyclic(edges | {tuple(s[0])})]

In [93]:
segments

[({0, 1}, 349.30401144305745),
 ({0, 2}, 391.0418681947875),
 ({0, 3}, 383.0204391314121),
 ({0, 4}, 199.89962990855176),
 ({0, 5}, 364.0447808729763),
 ({0, 6}, 338.8096401206489),
 ({0, 7}, 609.696280667211),
 ({0, 8}, 690.431902252931),
 ({0, 9}, 204.4253666122213),
 ({0, 10}, 183.29551097355267),
 ({0, 11}, 290.4028573916701),
 ({0, 12}, 136.70372473443274),
 ({0, 13}, 377.57973448944483),
 ({0, 14}, 301.99049212290043),
 ({0, 15}, 241.85490798374872),
 ({0, 16}, 258.5284939080245),
 ({0, 17}, 625.0755286998198),
 ({0, 18}, 401.9399754108604),
 ({0, 19}, 237.16017652663953),
 ({0, 20}, 401.4539129529872),
 ({0, 21}, 312.00387878900796),
 ({0, 22}, 438.9328915074273),
 ({0, 23}, 239.268808375456),
 ({0, 24}, 608.6754823442737),
 ({0, 25}, 287.5153079129395),
 ({0, 26}, 105.17159317964676),
 ({0, 27}, 139.23847120123253),
 ({0, 28}, 345.1318096169963),
 ({0, 29}, 196.77126979341847),
 ({0, 30}, 137.84822902835992),
 ({0, 31}, 635.9878049331178),
 ({0, 32}, 260.6010504730683),
 ({0, 3

In [94]:
cyclic(edges)

False

## Laboratorio 2 -  Soluzione proposta

In [95]:
POPULATION_SIZE = 100
GENERATIONS = 1000
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5
OFFSPRING_SIZE = 5

In [96]:
from dataclasses import dataclass

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

In [97]:
def valid_tsp(tsp):
    """
    It verifies the validity of a TSP route.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))

In [98]:
def inversion_mutation(tsp):
    tsp = np.array(tsp)
    value1, value2 = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[value1:value2+1] = np.flip(tsp[value1:value2+1])
    return tsp

In [99]:
# seleziono un genitore in base a quello che ha il costo minore 
def tournament_selection(population):
    tournament = population[np.random.choice(len(population), TOURNAMENT_SIZE, replace=False)]
    costs = np.array([tsp_cost(individual) for individual in tournament]) #seleziono i costi per ogni genitore
    return tournament[np.argmin(costs)]

In [100]:
# crossover tra due percorsi
def order_crossover(parent1, parent2):
    start, end = np.sort(np.random.choice(range(1, len(parent1) - 1), 2, replace=False))
    child = np.full(len(parent1), -1)
    child[start:end] = parent1[start:end]
    pos = end
    for city in parent2:
        if city not in child:
            child[pos] = city
            pos = (pos + 1) % (len(parent1) - 1) # necessario per non fare out of index
    child[-1] = child[0]
    return child

In [101]:
def inver_over(parent1, parent2):
    n = len(parent1)

    start, end = sorted(np.random.choice(range(n), 2, replace=False))
    child = parent1.copy()
    child[start:end+1] = parent2[start:end+1][::-1]
    for i in range(n):
        while child[i] in child[start:end+1]:
            for value in parent1:
                if value not in child:
                    child[i] = value
                    break

    return child

In [102]:
def create_initial_population():
    population = np.zeros((POPULATION_SIZE, len(CITIES) + 1), dtype=int)
    for i in range(POPULATION_SIZE):
        tsp = np.arange(len(CITIES))
        np.random.shuffle(tsp[1:])
        tsp = np.append(tsp, tsp[0])
        population[i] = tsp
    return population

## Algortimo lento ma preciso - genetic algorithm

In [103]:
def ea():
    population = create_initial_population()

    for generation in range(GENERATIONS):
        costs = np.array([tsp_cost(individual) for individual in population])
        best_index = np.argmin(costs)
        best_tsp = population[best_index]
        logging.info(f'Generation {generation} - Best cost: {tsp_cost(best_tsp):.2f}')

        new_population = np.zeros_like(population)
        new_population[0] = best_tsp

        offspring = []

        for _ in range(1, POPULATION_SIZE):
            if np.random.rand() < MUTATION_RATE:
                parent = tournament_selection(population)
                child = inversion_mutation(parent)
            else:
                parent1 = tournament_selection(population)
                parent2 = tournament_selection(population)
                child = order_crossover(parent1, parent2)
            
            offspring.append(child)  
            
        new_population[1:] = offspring[:POPULATION_SIZE - 1]
        population = new_population

    return best_tsp

best_path = ea()
ic(tsp_cost(best_path), tsp_cost.calls, valid_tsp(best_path), best_path)

INFO:root:Generation 0 - Best cost: 14899.92
INFO:root:Generation 1 - Best cost: 14581.71
INFO:root:Generation 2 - Best cost: 14050.00
INFO:root:Generation 3 - Best cost: 13180.18
INFO:root:Generation 4 - Best cost: 13180.18
INFO:root:Generation 5 - Best cost: 13040.66
INFO:root:Generation 6 - Best cost: 11968.93
INFO:root:Generation 7 - Best cost: 11968.93
INFO:root:Generation 8 - Best cost: 11413.11
INFO:root:Generation 9 - Best cost: 11156.59
INFO:root:Generation 10 - Best cost: 11156.59
INFO:root:Generation 11 - Best cost: 10932.76
INFO:root:Generation 12 - Best cost: 10423.82
INFO:root:Generation 13 - Best cost: 10423.82
INFO:root:Generation 14 - Best cost: 10423.82
INFO:root:Generation 15 - Best cost: 10294.96
INFO:root:Generation 16 - Best cost: 10294.96
INFO:root:Generation 17 - Best cost: 10025.29
INFO:root:Generation 18 - Best cost: 9833.53
INFO:root:Generation 19 - Best cost: 9764.10
INFO:root:Generation 20 - Best cost: 9692.55
INFO:root:Generation 21 - Best cost: 9122.95
IN

(np.float64(4174.935104213071),
 1040902,
 np.True_,
 array([ 2,  1, 11, 35, 21, 14, 15, 34, 39, 26, 27,  0, 33, 12, 30, 41, 43,
        23, 45,  5, 40, 44,  6,  3, 20, 18, 22, 42, 13, 28, 25, 32, 19,  9,
         4, 29, 10, 16, 36,  7, 24,  8, 37, 31, 17, 38,  2]))

## Algortimo veloce ma impreciso

In [115]:
NUM_CITIES = len(DIST_MATRIX)
MAX_STEPS = 10000
INITIAL_TEMPERATURE = 1000
COOLING_RATE = 0.985


def create_initial_solution():
    random = np.random.permutation(NUM_CITIES)
    solution = np.append(random, random[0])
    return solution

def single_mutation(solution: np.ndarray) -> np.ndarray:
    new_sol = solution.copy()
    i, j = np.random.randint(1, NUM_CITIES), np.random.randint(1, NUM_CITIES)
    while i == j:
        j = np.random.randint(1, NUM_CITIES)
        new_sol[i], new_sol[j] = new_sol[j], new_sol[i]
    return new_sol

def simulated_annealing():
    current_solution = create_initial_solution()
    best_solution = current_solution.copy()

    fitness_solution = tsp_cost(best_solution)
    history = [fitness_solution]       
    temperature = INITIAL_TEMPERATURE
    logging.info(f'Initial cost: {fitness_solution:.2f}')

    for step in tqdm(range(MAX_STEPS)):
        new_solution = single_mutation(current_solution)
        new_cost = tsp_cost(new_solution)
        history.append(new_cost) 
        cost_diff = new_cost - tsp_cost(current_solution)

        if cost_diff < 0 or np.random.random() < np.exp(-cost_diff / temperature):
            current_solution = new_solution.copy()
        if new_cost < fitness_solution:
            best_solution = new_solution.copy()
            fitness_solution = new_cost
        if temperature > 1e-17:
            temperature *= COOLING_RATE

    logging.info(f'Final cost: {fitness_solution:.2f}')
    return best_solution, history

best_path, cost_history = simulated_annealing()
ic(tsp_cost(best_path), tsp_cost.calls, valid_tsp(best_path), best_path)

INFO:root:Initial cost: 18340.82
100%|██████████| 1000/1000 [00:00<00:00, 36328.16it/s]
INFO:root:Final cost: 15983.69
ic| tsp_cost(best_path): np.float64(15983.687026308684)
    tsp_cost.calls: 1222926
    valid_tsp(best_path): np.True_
    best_path: array([22, 42, 33,  3, 43,  6, 12, 10,  8, 38, 36, 17, 26, 13, 29, 45,  5,
                      16, 19, 32, 15, 37, 41, 39, 35, 25, 14, 21, 11,  7,  9, 28,  4, 40,
                      23, 44, 18,  2, 31,  1, 27, 34,  0, 30, 20, 24, 22])


(np.float64(15983.687026308684),
 1222926,
 np.True_,
 array([22, 42, 33,  3, 43,  6, 12, 10,  8, 38, 36, 17, 26, 13, 29, 45,  5,
        16, 19, 32, 15, 37, 41, 39, 35, 25, 14, 21, 11,  7,  9, 28,  4, 40,
        23, 44, 18,  2, 31,  1, 27, 34,  0, 30, 20, 24, 22]))